<h1> Polymorphic Epitope Prediction </h1>

This tutorial illustrates how to use epytope to integrate genomic information of a patient for epitope prediction.

This tutorial entails:
- Variant construction
- Polymorphic Transcript/Protein/Peptide construction
- Polymorphic epitope prediction


In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from epytope.Core import Allele, Peptide, Protein,generate_peptides_from_proteins
from epytope.IO import read_lines, read_fasta
from epytope.EpitopePrediction import EpitopePredictorFactory
from epytope.Core import generate_transcripts_from_variants, generate_proteins_from_transcripts 
from epytope.Core import generate_peptides_from_variants
from epytope.IO import read_annovar_exonic
from epytope.IO import MartsAdapter
from epytope.IO import EIdentifierTypes

<h2> Chapter 1: Generating polymorphic epitopes </h2>
<br/>
We first have to construct variants to work with. We can either do this manually or by using one of the IO functions of epytope. Currently, epytope supports annotated exonic ANNOVAR files. Once the variant objects are created, we can use them to construct polymorphic transcripts with `epytope.Core.generate_transcript_from_variants`. For that we also have to specify from which database the additional transcript information (like sequence etc.) should be extracted and what kind of identification system (e.g. RefSeq, ENSEMBL) was used to annotate the variants. Here we use the `epytope.IO.MatrsAdapter` to connect to the remote BioMart DB and use `RefSeq` as indetification system via specifying this with `epytope.IO.EIdentifierTypes.REFSEQ`. We can also specify which of the community BioMart DB should be used instead of the central BioMart server with the named argument `biomart=`.<br/>

`epytope.Core.generate_transcript_from_variants` will generate all combinatorial possibilities of heterozygous and homozygous variants and incorporate these into the reference transcript sequence. This also means that the function becomes quickly unpractical once a large amount of heterozygous variants should be processed. $n$ heterozygous variants will generate $2^n$ transcript objects. This procedure is done, since no phasing information of the heterozygous variants are routinely available.


In [2]:
vars = read_annovar_exonic("./data/test_annovar.out")
mart =  MartsAdapter(biomart="http://grch37.ensembl.org/biomart/martservice?query=")
trans = generate_transcripts_from_variants(vars, mart, EIdentifierTypes.REFSEQ)

Once we generated the polymorphic transcripts, we can use `epytope.Core.generate_proteins_from_transcripts` to construct protein sequences. The so generated protein sequences will contain the non-synonymous variants that effected its protein sequence.

In [3]:
proteins = generate_proteins_from_transcripts(trans)

By using `epytope.Core.generate_peptides_from_proteins`, we can now generate polymorphic peptide objects from the previously generated polymorphic proteins. In addition to the proteins from which peptides are be generate, we have to specify a desired peptide length (e.g. 9-mers).

In [4]:
peptides1 = list(generate_peptides_from_proteins(proteins, 9))

/Users/antonia/PycharmProjects/Kfred/epytope/Core/Generator.py:57: UserWarning: For NM_001190266 bp does not match ref of assigned variant Variant(g.234183368A>G). Pos 645, var ref A, seq ref C 
  seq[pos]))
/Users/antonia/PycharmProjects/Kfred/epytope/Core/Generator.py:352: UserWarning: Transcript with ID NM_001293557 not found in DB
  warnings.warn("Transcript with ID %s not found in DB"%tId)
/Users/antonia/opt/miniconda3/envs/epy/lib/python3.7/site-packages/Bio/Seq.py:2338: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


If we are only interested in polymorphic peptides, or the high number of heterozygous variants prohibits the construction of all polymorphic transcripts/proteins, we can use `epytope.Core.generate_peptides_from_variants`. This function restricts the combinatorial exploration to a specific window size, thereby reducing the number of possible combination to $2^m$ with $m << n$. The window size represents the length of the desired peptides (e.g. 9-mers).

In [5]:
peptides2 = generate_peptides_from_variants(vars, 9, mart, EIdentifierTypes.REFSEQ)

/Users/antonia/PycharmProjects/Kfred/epytope/Core/Generator.py:246: UserWarning: Transcript with ID NM_001293557 not found in DB
  warnings.warn("Transcript with ID %s not found in DB"%tId)


**Note**: All function starting with `generate` are true generator functions. That means, they stall the calculations until the actual objects are needed, but can only be traversed once.

<h2> Chapter 2: Epitope prediction </h2>
<br/>
Once we have generated the peptide objects, we can for example predict their binding affinity. For that, we first have to initialize HLA allele objects, and an epitope prediction method. For more information see the <a href="https://github.com/KohlbacherLab/epytope/blob/master/epytope/tutorials/EpitopePrediction.ipynb">tutorial on epitope prediction</a>.


In [6]:
alleles = read_lines("./data/alleles.txt", in_type=Allele)
smm = EpitopePredictorFactory("smm")
pred_df = smm.predict(filter(lambda x: "*" not in str(x), peptides1), alleles=alleles)
pred_df.head()

,,HLA-A*02:01,HLA-B*15:01
Seq,Method,,
"(M, N, Q, V, T, I, Q, W, D)",smm,905962.037856,269153.480393
"(N, Q, V, T, I, Q, W, D, A)",smm,3540.870144,39719.154947
"(Q, V, T, I, Q, W, D, A, V)",smm,6793.756861,70631.755426
"(V, T, I, Q, W, D, A, V, I)",smm,1766.485187,1409.288798
"(T, I, Q, W, D, A, V, I, A)",smm,4276.711949,23713.737057


<h2> Chapter 3: Post-processing </h2>
<br/>
These polymorphic peptides have functionalities that allow the user to identify the variants that influenced the peptide sequences and locate their positions within the peptide sequence. With `Peptide.get_variants_by_protein()` we obtain a list of variants that influenced the peptide sequence originating from a specific protein. `Peptide.get_variants_by_protein_position()` returns a dictionary of where the key is the relative position of a variant within the peptide sequence that originated from a specific protein and protein position.

In [7]:
poly_peps = filter(lambda x: any(x.get_variants_by_protein(prot.transcript_id) for prot in x.get_all_proteins()) , peptides1)
c=0
for p in poly_peps:
    c+=1
    if c>=3:
        break
    for prot,poss in p.proteinPos.items():
        print()
        print(prot, p.get_variants_by_protein(prot))
        for pos in poss:
            vars = p.get_variants_by_protein_position(prot, pos)
            if vars:
                print(vars," Protein position: ",pos," Peptide: ",p)


NM_144701:epytope_1 [Variant(g.67705958G>A)]
{8: [Variant(g.67705958G>A)]}  Protein position:  372  Peptide:  IGIFNRSFQ

NM_144701:epytope_1 [Variant(g.67705958G>A)]
{7: [Variant(g.67705958G>A)]}  Protein position:  373  Peptide:  GIFNRSFQT
